In [47]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

os.chdir('/VOLUME/nia_vent_asynchrony')
from module.datasets import setup_data, set_dataloader, CustomDataset
from module.utils import load_and_stack_data
from model.VAE import VAE
from model.AsynchModel import AsynchModel
from module.metrics import calculate_any_metrics

In [2]:
from torchsummary import summary

In [3]:
vae = VAE().cpu()
summary(vae, input_size=(1, 480), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 16, 480]              96
       BatchNorm1d-2              [-1, 16, 480]              32
              ReLU-3              [-1, 16, 480]               0
         MaxPool1d-4              [-1, 16, 120]               0
            Conv1d-5              [-1, 32, 120]           2,592
       BatchNorm1d-6              [-1, 32, 120]              64
              ReLU-7              [-1, 32, 120]               0
         MaxPool1d-8               [-1, 32, 30]               0
            Conv1d-9               [-1, 64, 30]          10,304
      BatchNorm1d-10               [-1, 64, 30]             128
             ReLU-11               [-1, 64, 30]               0
           Linear-12                  [-1, 128]           3,968
           Linear-13                  [-1, 128]           3,968
           Linear-14                   

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


In [4]:
dat = load_and_stack_data(glob.glob('/VOLUME/nia_vent_asynchrony/data/processed_data/*_feature.csv'))

/VOLUME/nia_vent_asynchrony/data/processed_data/8c91bd20c48a51487ef5_feature.csv (25896, 481)
/VOLUME/nia_vent_asynchrony/data/processed_data/MICU01_feature.csv (38100, 481)
(63996, 481)


In [5]:
feature = dat.loc[:,dat.columns!='label'].values
label = dat['label'].values

In [6]:
feature.shape

(63996, 480)

In [7]:
train_dataloader, val_dataloader, test_dataloader = setup_data(feature, label)

X.shape (51196, 480) (12800, 480)
Y class distribution 0.07029846081725134 0.073984375


In [8]:
# train VAE
learning_rate = 1e-3
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bce_loss = torch.nn.BCELoss(reduction='mean')

def calculate_vae_loss(out, xi, batch_size):
    loss_r = bce_loss(out, xi)
    loss_kl = torch.mean(.5 * torch.sum(mu.pow(2) + torch.exp(logVar) - 1 - logVar, 1))
    loss = torch.mean(loss_r) + loss_kl
    return loss, loss_r, loss_kl


In [9]:
def calculate_bce_loss(out, target):
    loss = bce_loss(out, target)
    return loss

In [10]:

"""
Initialize the network and the Adam optimizer
"""
vae_model = VAE().to(device)
optimizer = torch.optim.Adam(vae_model.parameters(), lr=learning_rate)


"""
Training the network for a given number of epochs
The loss after every epoch is printed
"""
for epoch in range(num_epochs):
    print(f'Epoch {epoch}')
    for i_step, data in enumerate(train_dataloader):
        xi, _ = data
        xi = xi.to(device)
        batch_size = len(xi) 
        # Feeding a batch of images into the network to obtain the output image, mu, and logVar
        out, mu, logVar = vae_model(xi)

        # The loss is the BCE loss combined with the KL divergence to ensure the distribution is learnt
        # kl_divergence = 0.5 * torch.sum(-1 - logVar + mu.pow(2) + logVar.exp())
        # loss = F.binary_cross_entropy(out, xi, size_average=False) + kl_divergence
        loss, loss_r, loss_kl = calculate_vae_loss(out, xi, batch_size)

        # Backpropagation based on the loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('step', i_step, 'loss', loss.item(), 'loss_r', loss_r.item(), 'loss_kl', loss_kl.item())


Epoch 0


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


step 0 loss 19.12981605529785 loss_r 0.7170383930206299 loss_kl 18.412776947021484
step 1 loss 15.704118728637695 loss_r 0.6856671571731567 loss_kl 15.018451690673828
step 2 loss 13.391410827636719 loss_r 0.6638031601905823 loss_kl 12.727607727050781
step 3 loss 11.377877235412598 loss_r 0.6697244644165039 loss_kl 10.708152770996094
step 4 loss 10.184852600097656 loss_r 0.6520732045173645 loss_kl 9.532779693603516
step 5 loss 9.526041984558105 loss_r 0.6140482425689697 loss_kl 8.911993980407715
step 6 loss 7.846816062927246 loss_r 0.6441250443458557 loss_kl 7.202691078186035
step 7 loss 7.128811836242676 loss_r 0.6469420790672302 loss_kl 6.481869697570801
step 8 loss 6.042996406555176 loss_r 0.6532232165336609 loss_kl 5.389773368835449
step 9 loss 5.7660932540893555 loss_r 0.635067880153656 loss_kl 5.131025314331055
step 10 loss 4.868652820587158 loss_r 0.6725306510925293 loss_kl 4.196122169494629
step 11 loss 4.310087203979492 loss_r 0.6501337885856628 loss_kl 3.6599535942077637
step 

In [11]:
vae_model.eval()

VAE(
  (encoder_conv): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=same)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=same)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (decoder_conv): Sequential(
    (0): ConvTranspose1d(64, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Upsample(scale_factor=4.0, mode=linear)
    (4): 

In [12]:
# predict trainset and collect scores
train_score_list = []
for i_step, data in enumerate(train_dataloader):
    xi, target = data
    xi = xi.to(device)
    target = target.to(device)
    # Feeding a batch of images into the network to obtain the output image, mu, and logVar
    out, mu, logVar = vae_model(xi)

    loss, score, loss_kl = calculate_vae_loss(out, xi, batch_size)
    train_score_list.append(score.item())

In [13]:
np.percentile(train_score_list, [0, 25, 50, 75, 100], interpolation='nearest')

array([-2.10691833, -1.16776514, -0.92069262, -0.68554103, -0.04624803])

In [35]:
threshold = np.percentile(train_score_list, [99], interpolation='nearest')

In [36]:
threshold

array([-0.25783291])

In [17]:
sum(train_score_list>threshold)

4

In [18]:
# predict testset and find anomalies
score_list = []
for i_step, data in enumerate(test_dataloader):
    xi, target = data
    xi = xi.to(device)
    target = target.to(device)
    out, mu, logVar = vae_model(xi)

    loss, score, loss_kl = calculate_vae_loss(out, xi, batch_size)
    score_list.append(score.item())    

In [19]:
len(score_list)

12800

In [20]:
sum(score_list>threshold)

8214

In [21]:
testset_target = test_dataloader.dataset.y_data.squeeze(-1).squeeze(-1)

In [22]:
testset_target.shape

torch.Size([12800])

In [37]:
metrics = ['prec','recall','f1','f2','specificity',
                     'tn','fp','fn','tp',
                     'auroc','auprc']
calculate_any_metrics(testset_target, metrics, probs=torch.tensor(score_list), threshold=threshold.item())

{'auroc': 0.49,
 'auprc': 0.07,
 'prec': 0.07,
 'recall': 0.64,
 'f1': 0.13,
 'f2': 0.25,
 'specificity': 0.36,
 'tn': 4243.0,
 'fp': 7610.0,
 'fn': 343.0,
 'tp': 604.0,
 'auprcf1': 0.2}

In [38]:
abnomal_testset = test_dataloader.dataset[score_list>threshold]

In [48]:
abnomal_testset = CustomDataset((abnomal_testset[0]).numpy(), (abnomal_testset[1]).numpy())

In [49]:
vae_model.encoder_conv

Sequential(
  (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=same)
  (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=same)
  (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (8): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
  (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
)

In [50]:
ventdys_model = AsynchModel().to(device)


In [51]:
ventdys_model.load_state_dict(vae_model.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['classifier.1.weight', 'classifier.1.bias', 'classifier.1.running_mean', 'classifier.1.running_var', 'classifier.2.weight', 'classifier.2.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'], unexpected_keys=['decoder_conv.0.weight', 'decoder_conv.0.bias', 'decoder_conv.1.weight', 'decoder_conv.1.bias', 'decoder_conv.1.running_mean', 'decoder_conv.1.running_var', 'decoder_conv.1.num_batches_tracked', 'decoder_conv.4.weight', 'decoder_conv.4.bias', 'decoder_conv.5.weight', 'decoder_conv.5.bias', 'decoder_conv.5.running_mean', 'decoder_conv.5.running_var', 'decoder_conv.5.num_batches_tracked', 'decoder_conv.8.weight', 'decoder_conv.8.bias', 'decoder_conv.9.weight', 'decoder_conv.9.bias', 'decoder_conv.9.running_mean', 'decoder_conv.9.running_var', 'decoder_conv.9.num_batches_tracked', 'encFC1.weight', 'encFC1.bias', 'encFC2.weight', 'encFC2.bias', 'decFC1.weight', 'decFC1.bias'])

In [52]:
summary(ventdys_model, input_size=(1, 480), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 16, 480]              96
       BatchNorm1d-2              [-1, 16, 480]              32
              ReLU-3              [-1, 16, 480]               0
         MaxPool1d-4              [-1, 16, 120]               0
            Conv1d-5              [-1, 32, 120]           2,592
       BatchNorm1d-6              [-1, 32, 120]              64
              ReLU-7              [-1, 32, 120]               0
         MaxPool1d-8               [-1, 32, 30]               0
            Conv1d-9               [-1, 64, 30]          10,304
      BatchNorm1d-10               [-1, 64, 30]             128
             ReLU-11               [-1, 64, 30]               0
          Flatten-12                 [-1, 1920]               0
      BatchNorm1d-13                 [-1, 1920]           3,840
           Linear-14                   

In [53]:
ventdys_model.train()

AsynchModel(
  (encoder_conv): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=same)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=same)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): BatchNorm1d(1920, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=1920, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_featu

In [54]:
optimizer = torch.optim.Adam(ventdys_model.parameters(), lr=learning_rate)


"""
Training the network for a given number of epochs
The loss after every epoch is printed
"""
for epoch in range(num_epochs):
    print(f'Epoch {epoch}')
    for i_step, data in enumerate(train_dataloader):
        xi, target = data
        xi = xi.to(device)
        target = target.to(device).squeeze(-1).squeeze(-1).float()
        out = ventdys_model(xi)

        loss = calculate_bce_loss(out, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('step', i_step, 'loss', loss.item())
    print('epoch', epoch, 'loss', loss.item())


Epoch 0
epoch 0 loss 0.28029245138168335
Epoch 1
epoch 1 loss 0.17015030980110168
Epoch 2
epoch 2 loss 0.397563099861145
Epoch 3
epoch 3 loss 0.2782415747642517
Epoch 4
epoch 4 loss 0.34263134002685547


In [55]:
test_dataloader = set_dataloader(abnomal_testset, batch_size=128)

In [56]:
# predict testset and find anomalies
pred_list = []
for i_step, data in enumerate(test_dataloader):
    xi, target = data
    xi = xi.to(device).permute(0,2,1) # torch.Size([bs, 1, 480])
    target = target.to(device).squeeze(-1).squeeze(-1).float() # torch.Size([bs, 1])
    out = ventdys_model(xi)

    loss = calculate_bce_loss(out, target)
    pred_list.append(out.detach().cpu().numpy())
    

In [57]:
pred = np.concatenate(pred_list)
pred.shape

(8214,)

In [60]:
abnomal_testset.y_data.shape

torch.Size([8214, 1, 1])

In [61]:
testset_target = abnomal_testset.y_data.squeeze(-1).float()

In [62]:
metrics = ['prec','recall','f1','f2','specificity',
                     'tn','fp','fn','tp',
                     'auroc','auprc']
calculate_any_metrics(testset_target, metrics, probs=pred, threshold=0.3)

{'auroc': 0.51,
 'auprc': 0.07,
 'prec': 0.07,
 'recall': 0.21,
 'f1': 0.11,
 'f2': 0.15,
 'specificity': 0.8,
 'tn': 6050.0,
 'fp': 1560.0,
 'fn': 478.0,
 'tp': 126.0,
 'auprcf1': 0.18}